In [1]:
import pandas as pd

# データフレームを読み込む
df = pd.read_csv('USDJPY_TrailingStop2.csv', index_col='Date')
df.index = pd.to_datetime(df.index)
df.head()

,Open,High,Low,Close,ATR,Buy,Sell,Buy_cat,Sell_cat,Market
Date,,,,,,,,,,
2014-01-02 07:21:00,105.351,105.368,105.351,105.354,0.017400,-0.016200,0.034200,-1,1,-1
2014-01-02 07:22:00,105.355,105.355,105.355,105.355,0.016580,-0.014740,0.030740,-1,1,-1
2014-01-02 07:23:00,105.355,105.355,105.351,105.351,0.015951,-0.033853,0.032853,-1,1,-1
2014-01-02 07:24:00,105.354,105.355,105.349,105.355,0.015453,-0.036360,0.027360,-1,1,-1
2014-01-02 07:25:00,105.355,105.381,105.354,105.365,0.016031,-0.023092,0.019092,-1,1,-1


In [2]:
# 特徴量、目的変数
x = df.loc[:, 'Close']
y_buy = df.loc[:, 'Buy_cat']
y_sell = df.loc[:, 'Sell_cat']
# 訓練データ、テストデータに分割
x_train, x_test = x[x.index < '2018'], x[x.index >= '2018']
y_buy_train, y_buy_test = y_buy[y_buy.index < '2018'], y_buy[y_buy.index >= '2018']
y_sell_train, y_sell_test = y_sell[y_sell.index < '2018'], y_sell[y_sell.index >= '2018']
y_buy_test

Date
2018-01-02 07:00:00    1
2018-01-02 07:01:00    1
2018-01-02 07:02:00   -2
2018-01-02 07:03:00    1
2018-01-02 07:04:00    1
2018-01-02 07:05:00    1
2018-01-02 07:06:00    1
2018-01-02 07:07:00    1
2018-01-02 07:08:00    1
2018-01-02 07:09:00    1
2018-01-02 07:10:00   -1
2018-01-02 07:11:00   -1
2018-01-02 07:12:00   -1
2018-01-02 07:13:00   -1
2018-01-02 07:14:00   -1
2018-01-02 07:15:00   -1
2018-01-02 07:16:00   -1
2018-01-02 07:17:00   -1
2018-01-02 07:18:00   -1
2018-01-02 07:19:00   -2
2018-01-02 07:20:00    1
2018-01-02 07:21:00    1
2018-01-02 07:22:00    1
2018-01-02 07:23:00    1
2018-01-02 07:24:00    1
2018-01-02 07:25:00    1
2018-01-02 07:26:00    1
2018-01-02 07:27:00    1
2018-01-02 07:28:00    1
2018-01-02 07:29:00    1
                      ..
2018-10-06 05:29:00    1
2018-10-06 05:30:00    1
2018-10-06 05:31:00    1
2018-10-06 05:32:00    1
2018-10-06 05:33:00    1
2018-10-06 05:34:00    1
2018-10-06 05:35:00    1
2018-10-06 05:36:00    1
2018-10-06 05:37:00 

In [11]:
import numpy as np
from numba import jit
from keras.utils.np_utils import to_categorical

# 特徴量をClose(20)、目的変数をTrailingStop損益結果の4パターンとする
@jit
def gen_xy(x, y_buy, y_sell, window_len=20):
    X, Ybuy, Ysell = [], [], []
    for i in range(len(x) - window_len):
        Ybuy.append(y_buy[i + window_len - 1])
        Ysell.append(y_sell[i + window_len - 1])
        # 特徴量を正規化
        xx = x[i : i + window_len].copy()
        xx = xx - xx.min()
        xx = xx / xx.max()
        X.append(xx)
    X, Ybuy, Ysell = np.array(X), np.array(Ybuy), np.array(Ysell)
    # 目的変数を正規化
    Ybuy = np.where(Ybuy > 0, Ybuy + 1, Ybuy + 2)
    Ybuy = to_categorical(Ybuy.astype('int32'))
    Ysell = np.where(Ysell > 0, Ysell + 1, Ysell + 2)
    Ysell = to_categorical(Ysell.astype('int32'))
    return X, Ybuy, Ysell

Xtrain, Ytrain_buy, Ytrain_sell = gen_xy(x_train.values, y_buy_train.values, y_sell_train.values)
x_train.values.shape, x_train.values, Xtrain.shape, Xtrain, y_buy_train.values.shape, y_buy_train.values, Ytrain_buy.shape, Ytrain_buy

((1491718,),
 array([105.354, 105.355, 105.351, ..., 112.671, 112.668, 112.658]),
 (1491698, 20),
 array([[0.59259259, 0.62962963, 0.48148148, ..., 0.18518519, 0.11111111,
         0.25925926],
        [0.62962963, 0.48148148, 0.62962963, ..., 0.11111111, 0.25925926,
         0.11111111],
        [0.48148148, 0.62962963, 1.        , ..., 0.25925926, 0.11111111,
         0.51851852],
        ...,
        [0.18181818, 0.40909091, 0.63636364, ..., 0.59090909, 0.68181818,
         0.5       ],
        [0.40909091, 0.63636364, 0.63636364, ..., 0.68181818, 0.5       ,
         0.40909091],
        [0.63636364, 0.63636364, 0.63636364, ..., 0.5       , 0.40909091,
         0.27272727]]),
 (1491718,),
 array([-1, -1, -1, ..., -1, -1, -1], dtype=int64),
 (1491698, 4),
 array([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]], dtype=float32))

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

''' CNNで買い専用AIの訓練 '''

model = Sequential()
model.add(Conv1D(32, 3, activation='relu', padding='valid', input_shape=(20, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, 3, activation='relu', padding='valid'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

early_stopping = EarlyStopping(monitor='val_acc', mode='auto', patience=8)
model_checkpoint = ModelCheckpoint(filepath="20.Buyer4.h5")

# 訓練実行
model.fit(Xtrain.reshape(Xtrain.shape[0], 20, 1), Ytrain_buy, batch_size=2048, epochs=256, shuffle=True,
          validation_split=0.1, # 訓練データのうち10％を検証データとして仕様
          callbacks=[early_stopping, model_checkpoint]
         )

Train on 1342528 samples, validate on 149170 samples
Epoch 1/256
1342528/1342528 [==============================] - 7s 5us/step - loss: 1.2188 - acc: 0.4819 - val_loss: 1.2112 - val_acc: 0.4877
Epoch 2/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 1.2166 - acc: 0.4822 - val_loss: 1.2116 - val_acc: 0.4877
Epoch 3/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 1.2165 - acc: 0.4822 - val_loss: 1.2112 - val_acc: 0.4877
Epoch 4/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 1.2164 - acc: 0.4822 - val_loss: 1.2111 - val_acc: 0.4877
Epoch 5/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 1.2163 - acc: 0.4822 - val_loss: 1.2116 - val_acc: 0.4877
Epoch 6/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 1.2162 - acc: 0.4822 - val_loss: 1.2108 - val_acc: 0.4877
Epoch 7/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 1.2161 - acc: 0.48

In [16]:
''' 目的変数を「勝てる・負ける」の2パターンにして学習（買いAI） '''

Y = np.argmax(Ytrain_buy, axis=1) # 目的変数を0～3のカテゴリに戻す
Y = np.where(Y > 1, 1, 0) # 売買結果を「勝てる・負ける」の2パターンに変換
Y = to_categorical(Y.astype('int32'))

model = Sequential()
model.add(Conv1D(32, 3, activation='relu', padding='valid', input_shape=(20, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, 3, activation='relu', padding='valid'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

early_stopping = EarlyStopping(monitor='val_acc', mode='auto', patience=8)
model_checkpoint = ModelCheckpoint(filepath="20.Buyer2.h5")

# 訓練実行
model.fit(Xtrain.reshape(Xtrain.shape[0], 20, 1), Y, batch_size=2048, epochs=256, shuffle=True,
          validation_split=0.1, # 訓練データのうち10％を検証データとして仕様
          callbacks=[early_stopping, model_checkpoint]
         )

Train on 1342528 samples, validate on 149170 samples
Epoch 1/256
1342528/1342528 [==============================] - 7s 5us/step - loss: 0.6634 - acc: 0.6217 - val_loss: 0.6627 - val_acc: 0.6245
Epoch 2/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6631 - acc: 0.6220 - val_loss: 0.6618 - val_acc: 0.6245
Epoch 3/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6630 - acc: 0.6220 - val_loss: 0.6618 - val_acc: 0.6245
Epoch 4/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6630 - acc: 0.6220 - val_loss: 0.6617 - val_acc: 0.6245
Epoch 5/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6629 - acc: 0.6220 - val_loss: 0.6618 - val_acc: 0.6245
Epoch 6/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6629 - acc: 0.6220 - val_loss: 0.6619 - val_acc: 0.6245
Epoch 7/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6628 - acc: 0.62

In [17]:
''' CNNで売り専用AIの訓練 '''

model = Sequential()
model.add(Conv1D(32, 3, activation='relu', padding='valid', input_shape=(20, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, 3, activation='relu', padding='valid'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

early_stopping = EarlyStopping(monitor='val_acc', mode='auto', patience=8)
model_checkpoint = ModelCheckpoint(filepath="20.Seller4.h5")

# 訓練実行
model.fit(Xtrain.reshape(Xtrain.shape[0], 20, 1), Ytrain_sell, batch_size=2048, epochs=256, shuffle=True,
          validation_split=0.1, # 訓練データのうち10％を検証データとして仕様
          callbacks=[early_stopping, model_checkpoint]
         )

Train on 1342528 samples, validate on 149170 samples
Epoch 1/256
1342528/1342528 [==============================] - 7s 5us/step - loss: 0.9212 - acc: 0.6300 - val_loss: 0.9045 - val_acc: 0.6377
Epoch 2/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.9149 - acc: 0.6307 - val_loss: 0.9051 - val_acc: 0.6377
Epoch 3/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.9145 - acc: 0.6307 - val_loss: 0.9045 - val_acc: 0.6377
Epoch 4/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.9144 - acc: 0.6307 - val_loss: 0.9049 - val_acc: 0.6377
Epoch 5/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.9143 - acc: 0.6307 - val_loss: 0.9046 - val_acc: 0.6377
Epoch 6/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.9142 - acc: 0.6307 - val_loss: 0.9043 - val_acc: 0.6377
Epoch 7/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.9142 - acc: 0.63

In [18]:
''' 目的変数を「勝てる・負ける」の2パターンにして学習（売りAI） '''

Y = np.argmax(Ytrain_sell, axis=1) # 目的変数を0～3のカテゴリに戻す
Y = np.where(Y > 1, 1, 0) # 売買結果を「勝てる・負ける」の2パターンに変換
Y = to_categorical(Y.astype('int32'))

model = Sequential()
model.add(Conv1D(32, 3, activation='relu', padding='valid', input_shape=(20, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, 3, activation='relu', padding='valid'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

early_stopping = EarlyStopping(monitor='val_acc', mode='auto', patience=8)
model_checkpoint = ModelCheckpoint(filepath="20.Seller2.h5")

# 訓練実行
model.fit(Xtrain.reshape(Xtrain.shape[0], 20, 1), Y, batch_size=2048, epochs=256, shuffle=True,
          validation_split=0.1, # 訓練データのうち10％を検証データとして仕様
          callbacks=[early_stopping, model_checkpoint]
         )

Train on 1342528 samples, validate on 149170 samples
Epoch 1/256
1342528/1342528 [==============================] - 7s 5us/step - loss: 0.6589 - acc: 0.6305 - val_loss: 0.6545 - val_acc: 0.6378
Epoch 2/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6585 - acc: 0.6308 - val_loss: 0.6547 - val_acc: 0.6378
Epoch 3/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6585 - acc: 0.6308 - val_loss: 0.6546 - val_acc: 0.6378
Epoch 4/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6584 - acc: 0.6308 - val_loss: 0.6554 - val_acc: 0.6378
Epoch 5/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6584 - acc: 0.6308 - val_loss: 0.6548 - val_acc: 0.6378
Epoch 6/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6583 - acc: 0.6308 - val_loss: 0.6546 - val_acc: 0.6378
Epoch 7/256
1342528/1342528 [==============================] - 6s 5us/step - loss: 0.6583 - acc: 0.63